## Download the dataset



In [5]:
!git clone https://github.com/mttgdd/RobotCarDataset-Scraper.git
!mkdir ./Downloads

Cloning into 'RobotCarDataset-Scraper'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 46.33 KiB | 817.00 KiB/s, done.
Resolving deltas: 100% (108/108), done.


In [6]:
!python RobotCarDataset-Scraper/scrape_mrgdatashare.py --downloads_dir Downloads --datasets_file RobotCarDataset-Scraper/demo.csv --username zirui --password 2251598zzrrywD

ScrapeMRGDatashare is starting...
reading datasets_file: /content/RobotCarDataset-Scraper/demo.csv
got num_datasets: 2
got csrf_middleware_token: I4a3oNYslMpBO1JO3lcoNX1JXjQ9xECZOjpowPCY39myLI9IP5Y9yIUVu7tbio0b
Logged in!
num_chunks_in_period: 0, period_seconds: 600
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-10-29-12-18-17/2015-10-29-12-18-17_tags.tar
downloading local_file_path: /content/Downloads/2015-10-29-12-18-17_tags.tar
100% 10.0/10.0 [00:00<00:00, 31.1kKB/s]
unzipping local_file_path: /content/Downloads/2015-10-29-12-18-17_tags.tar
requesting file_url: http://mrgdatashare.robots.ox.ac.uk:80/download/?filename=datasets/2015-10-29-12-18-17/2015-10-29-12-18-17_stereo_left_01.tar
downloading local_file_path: /content/Downloads/2015-10-29-12-18-17_stereo_left_01.tar
100% 4.46M/4.46M [03:19<00:00, 22.3kKB/s]
unzipping local_file_path: /content/Downloads/2015-10-29-12-18-17_stereo_left_01.tar
requesting file_url: http://mrgdatashare.ro

## Extract descriptor



In [0]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image


model_file = 'alexnet_places365.pth.tar'
if not os.access(model_file, os.W_OK):
    os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)


# from functools import partial
# import pickle
# pickle.load = partial(pickle.load, encoding="latin1")
# pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
# model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

model = models.alexnet(num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

In [0]:
import torch.nn as nn
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

def myforward(model, x):
    m1 = nn.Sequential(*list(model.children())[:-1])
    m2 = nn.Sequential(*list(model.children())[2][:2])
    feature = m1(x)
    feature = torch.flatten(feature, start_dim=0, end_dim=-1)
    feature = m2(feature)
    feature = feature.view(x.size(0), -1)
    return feature


# forward image
tf = returnTF()

In [0]:
x = "123.png"
savepath = "/content/2014-12-09-13-21-02/"
savepath+x[:-4]+'.mat'

'/content/2014-12-09-13-21-02/123.mat'

In [0]:
import scipy.io as io
import os
filepath = "/content/Downloads/2015-10-29-12-18-17/stereo/left/"
savepath = "/content/2015-10-29-12-18-17/"
 
for filename in os.listdir(filepath):
    img = cv2.imread(filepath+filename,cv2.IMREAD_GRAYSCALE)
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    input_img = V(tf(img).unsqueeze(0))
    output = myforward(model, input_img)
    x = output.data.numpy()
    io.savemat(savepath+filename[:-4]+'.mat', {'fc6': x}) 


In [0]:
!zip -r 2015-10-29-12-18-17.zip /content/2015-10-29-12-18-17/

In [0]:
from google.colab import files
files.download('2014-12-10-18-10-50.zip')
files.download('2014-12-16-09-14-09.zip')
files.download('2015-02-03-08-45-10.zip')
files.download('2015-05-19-14-06-38.zip')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!rm -rf /content/Downloads/2014-12-09-13-21-02

# NetVLAD 

In [3]:
!git clone https://github.com/uzh-rpg/netvlad_tf_open.git

Cloning into 'netvlad_tf_open'...
remote: Enumerating objects: 50, done.
remote: Total 50 (delta 0), reused 0 (delta 0), pack-reused 50
Unpacking objects: 100% (50/50), done.


In [4]:
!wget -P ./checkpoints http://rpg.ifi.uzh.ch/datasets/netvlad/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip

--2020-04-28 13:48:49--  http://rpg.ifi.uzh.ch/datasets/netvlad/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip
Resolving rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)... 130.60.155.125
Connecting to rpg.ifi.uzh.ch (rpg.ifi.uzh.ch)|130.60.155.125|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1108966217 (1.0G) [application/zip]
Saving to: ‘./checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip’

vd16_pitts30k_conv5 100%[===================>]   1.03G  9.96MB/s    in 1m 47s  

2020-04-28 13:50:37 (9.88 MB/s) - ‘./checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip’ saved [1108966217/1108966217]



In [5]:
!unzip ./checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip 

Archive:  ./checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.zip
   creating: vd16_pitts30k_conv5_3_vlad_preL2_intra_white/
  inflating: vd16_pitts30k_conv5_3_vlad_preL2_intra_white/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.data-00000-of-00001  
  inflating: vd16_pitts30k_conv5_3_vlad_preL2_intra_white/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.index  
  inflating: vd16_pitts30k_conv5_3_vlad_preL2_intra_white/vd16_pitts30k_conv5_3_vlad_preL2_intra_white.meta  


In [0]:
!rm -rf ./checkpoints

In [1]:
%tensorflow_version 1.x
import tensorflow 
tensorflow.__version__

TensorFlow 1.x selected.


'1.15.2'

In [4]:
!ls ./checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white

vd16_pitts30k_conv5_3_vlad_preL2_intra_white.data-00000-of-00001
vd16_pitts30k_conv5_3_vlad_preL2_intra_white.index
vd16_pitts30k_conv5_3_vlad_preL2_intra_white.meta


In [3]:
import cv2
import numpy as np
import tensorflow as tf

import netvlad_tf.net_from_mat as nfm
import netvlad_tf.nets as nets

tf.reset_default_graph()

image_batch = tf.placeholder(
        dtype=tf.float32, shape=[None, None, None, 3])

net_out = nets.vgg16NetvladPca(image_batch)
saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, nets.defaultCheckpoint())

inim = cv2.imread('example.jpg')
inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)

batch = np.expand_dims(inim, axis=0)
result = sess.run(net_out, feed_dict={image_batch: batch})

INFO:tensorflow:Restoring parameters from /content/checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white


In [0]:
import scipy.io as io
import os
io.savemat('test.mat', {'netvlad': result}) 

In [8]:
import cv2
import numpy as np
import tensorflow as tf

import netvlad_tf.net_from_mat as nfm
import netvlad_tf.nets as nets

tf.reset_default_graph()

image_batch = tf.placeholder(
        dtype=tf.float32, shape=[None, None, None, 3])

net_out = nets.vgg16NetvladPca(image_batch)
saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, nets.defaultCheckpoint())



import scipy.io as io
import os
filepath = "/content/Downloads/2015-10-29-12-18-17/stereo/left/"
savepath = "/content/2015-10-29-12-18-17/"
 
for filename in os.listdir(filepath):
    inim = cv2.imread(filepath+filename, cv2.IMREAD_GRAYSCALE)
    inim = cv2.cvtColor(inim, cv2.COLOR_GRAY2RGB)
    batch = np.expand_dims(inim, axis=0)
    x = sess.run(net_out, feed_dict={image_batch: batch})
    io.savemat(savepath+filename[:-4]+'.mat', {'netvlad': x}) 

INFO:tensorflow:Restoring parameters from /content/checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white


In [9]:
!zip -r 2015-10-29-12-18-17.zip /content/2015-10-29-12-18-17/

Streaming output truncated to the last 5000 lines.
  adding: content/2015-10-29-12-18-17/1446121441070682.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121181755722.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121327069588.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121413382912.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121392570209.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121420382978.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121177255681.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121336632180.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121155755483.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121483758597.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121407695359.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121112005092.mat (deflated 8%)
  adding: content/2015-10-29-12-18-17/1446121416195439.mat (deflated 8%)


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
